In [1]:
import os
import numpy as np
import pandas as pd
import time
import os.path as osp

In [2]:
import gc 
# del a
gc.collect()

20

# Stage-1：Data access

In [3]:
import DataHelper
import importlib
from collections import Counter
importlib.reload(DataHelper)

<module 'DataHelper' from '/home/ubuntu/DBRF-master/DataHelper.py'>

In [4]:
# X_train,y_train, X_sub, y_sub = DataHelper.get_driver_data()
# X_train,y_train, X_sub, y_sub = DataHelper.get_yeast_data()

In [5]:
from keras.datasets import mnist
(X_train_, y_train), (X_test_, y_sub) = mnist.load_data()
X_train = np.zeros((X_train_.shape[0],784))
for i in range(X_train_.shape[0]):
    X_train[i] = X_train_[i].flatten()

X_sub = np.zeros((X_test_.shape[0],784))
for i in range(X_test_.shape[0]):
    X_sub[i] = X_test_[i].flatten()


Using TensorFlow backend.


# Stage-2：Model application

In [6]:
import ModelUtils
import importlib
from collections import Counter
importlib.reload(ModelUtils)

<module 'ModelUtils' from '/home/ubuntu/DBRF-master/ModelUtils.py'>

In [7]:
from sklearn import metrics
def acc_metrix_mult(a, p):
#     print(p)
    return "acc", metrics.accuracy_score(a, p)

# DT, RF, GDBT, XGB 

In [10]:
ModelUtils.decision_tree(X_train, y_train, X_sub, y_sub, feval=acc_metrix_mult)

train ('acc', 1.0)
test ('acc', 0.8787)


(DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                        max_features=None, max_leaf_nodes=None,
                        min_impurity_decrease=0.0, min_impurity_split=None,
                        min_samples_leaf=1, min_samples_split=2,
                        min_weight_fraction_leaf=0.0, presort=False,
                        random_state=None, splitter='best'),
 array([7, 2, 1, ..., 4, 5, 6], dtype=uint8))

In [11]:
ModelUtils.random_forest(X_train, y_train, X_sub, y_sub, n_estimators=1000, feval=acc_metrix_mult)

start train: 2020-02-11 13:21:51.021637
start test: 2020-02-11 13:23:57.710560
end test: 2020-02-11 13:23:59.158252
test ('acc', 0.9717)
train ('acc', 1.0)


(RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                        max_depth=None, max_features='auto', max_leaf_nodes=None,
                        min_impurity_decrease=0.0, min_impurity_split=None,
                        min_samples_leaf=1, min_samples_split=2,
                        min_weight_fraction_leaf=0.0, n_estimators=1000,
                        n_jobs=-1, oob_score=False, random_state=None, verbose=0,
                        warm_start=False),
 array([7, 2, 1, ..., 4, 5, 6], dtype=uint8))

In [ ]:
ModelUtils.gdbt_model(X_train, y_train, X_sub, y_sub, n_estimators=1000, feval=acc_metrix_mult)

In [ ]:
ModelUtils.xgb_model(X_train, y_train, X_sub, y_sub, num_class=26, feval=acc_metrix_mult)

# BaseLine

In [17]:
ModelUtils.baseline_1_flod(X_train, y_train, X_sub, y_sub, num_class=26, feval=acc_metrix_mult)

X_train.shape, y_train.shape:(60000, 784)(60000,)
X_valid.shape, y_valid.shape:(10000, 784)(10000,)


KeyboardInterrupt: 

In [ ]:
gc.collect()

In [ ]:
ModelUtils.baseline_model(X_train, y_train, X_sub, y_sub, num_class=26, feval=acc_metrix_mult)

# Stage-3：EcoForest

In [13]:
import EnhancedForest_multiclass
import EnhancedForest_producer
import DecomposerForest
import LogUtils
import AlgorithmUtils
import importlib
importlib.reload(DecomposerForest)
importlib.reload(LogUtils)
importlib.reload(EnhancedForest_multiclass)
importlib.reload(EnhancedForest_producer)
importlib.reload(AlgorithmUtils)
np.seterr(divide='ignore', invalid='ignore')
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.filterwarnings("ignore", category=RuntimeWarning) 

In [ ]:
decoForest = DecomposerForest.DecomposerForest(X_train, y_train, X_sub, y_sub, num_class=10, flag="mnist", isLRStacker=False)
decoForest.set_parameter(criterion="gini", dropout=None, min_samples_leaf=1)
decoForest.fit(n_estimators=150000, kfold=3, feval=acc_metrix_mult)


2020-02-11 13:25:41 layer: 1


In [ ]:
np.max(np.array(decoForest.test_loss_lt)),list(enumerate(decoForest.test_loss_lt) )

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10,10))
x = range(len(decoForest.train_loss_lt))

plt.subplot(221)
plt.plot(x[1:],decoForest.pass_train_loss_lt[1:], label="train-pass", color="blue")
plt.plot(x[1:],decoForest.pass_vaild_loss_lt[1:], label="vaild-pass", color="green")
plt.plot(x[1:],decoForest.pass_test_loss_lt[1:], label="test-pass", color="red")
plt.legend()
plt.subplot(222)
plt.plot(x[1:],decoForest.pass_train_loss_lt_now[1:], label="train-pass-now", color="blue")
plt.plot(x[1:],decoForest.pass_vaild_loss_lt_now[1:], label="vaild-pass-now", color="green")
plt.plot(x[1:],decoForest.pass_test_loss_lt_now[1:], label="test-pass-now", color="red")
plt.legend()
plt.subplot(223)
plt.plot(x,decoForest.train_loss_lt, label="train-loss", color="blue")
plt.plot(x,decoForest.vaild_loss_lt, label="vaild-loss", color="green")
plt.plot(x,decoForest.test_loss_lt, label="test-loss", color="red")
plt.legend()
plt.subplot(224)
plt.plot(x,decoForest.pass_data_rate_lt[:], label="data-pass-rate")
plt.legend()

plt.show()

In [ ]:
pass_train_x_list = decoForest.enhancedDTree.pass_data_x_list
pass_train_y_list = decoForest.enhancedDTree.pass_data_y_list
pass_test_x_list = decoForest.enhancedDTree.pass_data_test_list
pass_test_y_list = decoForest.enhancedDTree.pass_real_test_list

In [ ]:
producerForest = EnhancedForest_producer.Producer(pass_train_x_list, pass_train_y_list, pass_test_x_list, pass_test_y_list)
producerForest.prodect(26, acc_metrix_mult)

In [ ]:
producerForest.train_x.shape

In [ ]:
Counter(producerForest.train_y)

In [ ]:
tmp = np.array(list(Counter(producerForest.train_y).values()))
tmp

In [ ]:
tmp2 = np.array(list(Counter(producerForest.train_y).keys()))
tmp2

In [ ]:
len(np.where(tmp == 1)[0]) == 0

In [ ]:
index_tmp = np.where(tmp == 1)[0]
tmp2[index_tmp]

In [ ]:
np.where(producerForest.train_y == 3)[0][0]

In [ ]:
producerForest.train_y[105]

In [ ]:
producerForest.train_x[105]

In [ ]:
producerForest.train_y.shape

In [ ]:
baseline_model(producerForest.train_x, producerForest.train_y, producerForest.test_x, producerForest.test_y, num_class=26)

In [ ]:
a = [0, 3, 8, 9, 11]
b = [0, 11, 8, 9]

In [ ]:
[a.index(i) for i in b]

In [ ]:
x_index = list(test_index)

In [ ]:
x_index

In [ ]:
p_all_fold_prob[:, y_index][x_index] = y_pred_prob

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from scipy import stats
def baseline_model(X_train, y_train, X_sub, y_sub, num_class, feval=metrics.accuracy_score):
    kfold = 10
    X = X_train.copy()
    y = y_train.copy()
    X_test = X_sub.copy()
    real_y = y_sub.copy()
    max_depth = None
    random_state = 1023
    min_samples_leaf = 1
    criterion = 'entropy'#'gini'
    # criterion = 'gini'#'gini'
    n_estimators = 200

    sss = StratifiedKFold(n_splits=kfold, shuffle=True, random_state=9487)

    print("X_train.shape, y_train.shape:"+str(X.shape)+str(y.shape))
    
    
    tmp1 = np.array(list(Counter(y_train).values()))
    tmp2 = np.array(list(Counter(y_train).keys()))
    index_tmp = np.where(tmp1 == 1)[0]
    for i in index_tmp:
        assert len(index_tmp) > 0
        tmp2_index = i
        data_key = tmp2[tmp2_index]
        tmp_data_index = np.where(producerForest.train_y == data_key)[0][0]
        X = np.delete(X, tmp_data_index, axis=0)
        y = np.delete(y, tmp_data_index, axis=0)
    print("X_train.shape, y_train.shape:"+str(X.shape)+str(y.shape))
        
    p_all_fold = np.zeros(X.shape[0])
    p_all_fold_prob = np.zeros((X.shape[0], len(set(y))))
#     p_all_fold_prob = np.zeros((X.shape[0], num_class))
    p_all_fold_prob_test = np.zeros((X_test.shape[0], len(set(y))))
#     p_all_fold_prob_test = np.zeros((X_test.shape[0], num_class))
    clf_folds = []
    
    ###### 
#     if len(set(y_train)) != num_class:
    y_index = list(set(y_train))
    y_index.sort()
    print(y_index)
        
    
    for i, (train_index, test_index) in enumerate(sss.split(X, y)):
        X_train_bl, X_valid_bl = X[train_index], X[test_index]
        y_train_bl, y_valid_bl = y[train_index], y[test_index]
            
        print("X_train.shape, y_train.shape:"+str(X_train_bl.shape)+str(y_train_bl.shape))
        print("X_valid.shape, y_valid.shape:"+str(X_valid_bl.shape)+str(y_valid_bl.shape))
            
        clf = RandomForestClassifier(max_depth=max_depth, random_state=random_state, 
                                        min_samples_leaf=min_samples_leaf, n_estimators=n_estimators, n_jobs=-1,
                                             #max_leaf_nodes=100,
        #                                  n_estimators=2, n_jobs=8, oob_score=True, verbose=1, boostrap=False,
                                         criterion=criterion)
        print(len(set(y_train_bl)))
        clf = clf.fit(X_train_bl, y_train_bl)
        clf_folds.append(clf)
        # print(clf)
        
        cross_score = cross_val_score(clf, X_valid_bl, y_valid_bl, cv=3)
        print("    cross_score: %.5f" % (cross_score.mean()))
        print("    test score", feval(real_y, clf.predict(X_test)))
        
        
#         if len(set(y_train_bl)) < len(set(y_train)):
#             tmp1 = np.array(list(Counter(y_train).values()))
#             tmp2 = np.array(list(Counter(y_train).keys()))
#             index_tmp = np.where(tmp1 == 1)[0]
#             assert len(index_tmp) > 0
#             add_x = []
#             add_y = []
#             for i in range(len(index_tmp) == 0):
#                 data_key = tmp2[index_tmp]
#                 tmp_data_index = np.where(producerForest.train_y == data_key)[0][0]
#                 add_x.append(X_train[tmp_data_index])
#                 add_y.append(y_train[tmp_data_index])
                
        
        
        if not num_class:
            y_pred = clf.predict_proba(X_valid_bl)[:,1]
        else:
            y_pred = clf.predict(X_valid_bl)
            y_pred_prob = clf.predict_proba(X_valid_bl)
            y_pred_prob_test = clf.predict_proba(X_test)
            
#             if len(set(y_train_bl)) < len(set(y_train)):
#                 y_index_fold = list(set(y_train_bl))
#                 y_index_fold.sort()
# #                 print(y_index, y_index_fold)
#                 y_fold_index = [y_index.index(i) for i in y_index_fold]
#             else:
#                 y_fold_index = list(range(len(y_index)))
                

        p_all_fold[test_index] += y_pred
#         p_all_fold_prob[:, y_index][test_index] += y_pred_prob
#         p_all_fold_prob_test[:, y_index] += y_pred_prob_test/kfold
#         p_all_fold_prob[:, y_fold_index][test_index] += y_pred_prob
#         p_all_fold_prob_test[:, y_fold_index] += y_pred_prob_test/kfold
        print(test_index.shape, y_pred_prob.shape, p_all_fold_prob[test_index, :].shape)
        p_all_fold_prob[test_index, :] += y_pred_prob
        p_all_fold_prob_test[:, :] += y_pred_prob_test/kfold
    print("train kflod pred:", feval(y, p_all_fold))

    #### train pred
    y_preds = np.zeros((X.shape[0], len(clf_folds)))
    for i, clf in enumerate(clf_folds):
        if not num_class:
            y_pred = clf.predict_proba(X)[:,1]
        else:
            y_pred = clf.predict(X)
        y_preds[:, i] = y_pred

    if not num_class:
        result = y_preds.mean(axis=1)
    else:
    #     print(y_preds)
        m = stats.mode(y_preds, axis=1)
        result = np.array([i[0] for i in m[0]])
    print("train pred", feval(y, result))

    #### test pred
    y_preds = np.zeros((X_test.shape[0], len(clf_folds)))
    for i, clf in enumerate(clf_folds):
        if not num_class:
            y_pred = clf.predict_proba(X_test)[:,1]
        else:
            y_pred = clf.predict(X_test)
        y_preds[:, i] = y_pred

    if not num_class:
        result = y_preds.mean(axis=1)
    else:
    #     print(y_preds)
        m = stats.mode(y_preds, axis=1)
        result = np.array([i[0] for i in m[0]])
    # print(result)
    print("test pred", feval(real_y, result))

    ### lr pred
    log_model = LogisticRegression()
    log_model.fit(p_all_fold_prob, y)
    res = log_model.predict(p_all_fold_prob)
    print("lr train", feval(y, res))
    res = log_model.predict(p_all_fold_prob_test)
    print("lr test", feval(real_y, res))
    print("lr coef_", np.mean(log_model.coef_))
    return clf_folds

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10,10))
x = range(len(decoForest.train_loss_lt))

plt.subplot(221)
plt.plot(x[1:],decoForest.pass_train_loss_lt[1:], label="train-pass", color="blue")
plt.plot(x[1:],decoForest.pass_vaild_loss_lt[1:], label="vaild-pass", color="green")
plt.plot(x[1:],decoForest.pass_test_loss_lt[1:], label="test-pass", color="red")
plt.legend()
plt.subplot(222)
plt.plot(x[1:],decoForest.pass_train_loss_lt_now[1:], label="train-pass-now", color="blue")
plt.plot(x[1:],decoForest.pass_vaild_loss_lt_now[1:], label="vaild-pass-now", color="green")
plt.plot(x[1:],decoForest.pass_test_loss_lt_now[1:], label="test-pass-now", color="red")
plt.legend()
plt.subplot(223)
plt.plot(x,decoForest.train_loss_lt, label="train-loss", color="blue")
plt.plot(x,decoForest.vaild_loss_lt, label="vaild-loss", color="green")
plt.plot(x,decoForest.test_loss_lt, label="test-loss", color="red")
plt.legend()
plt.subplot(224)
plt.plot(x,decoForest.pass_data_rate_lt[:], label="data-pass-rate")
plt.legend()

plt.show()

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10,10))
x = range(len(decoForest.train_loss_lt))

plt.subplot(221)
plt.plot(x[1:],decoForest.pass_train_loss_lt[1:], label="train-pass", color="blue")
plt.plot(x[1:],decoForest.pass_vaild_loss_lt[1:], label="vaild-pass", color="green")
plt.plot(x[1:],decoForest.pass_test_loss_lt[1:], label="test-pass", color="red")
plt.legend()
plt.subplot(222)
plt.plot(x[1:],decoForest.pass_train_loss_lt_now[1:], label="train-pass-now", color="blue")
plt.plot(x[1:],decoForest.pass_vaild_loss_lt_now[1:], label="vaild-pass-now", color="green")
plt.plot(x[1:],decoForest.pass_test_loss_lt_now[1:], label="test-pass-now", color="red")
plt.legend()
plt.subplot(223)
# plt.plot(x,decoForest.train_loss_lt, label="train-loss", color="blue")
plt.plot(x,decoForest.vaild_loss_lt, label="vaild-loss", color="green")
plt.plot(x,decoForest.test_loss_lt, label="test-loss", color="red")
plt.legend()
plt.subplot(224)
plt.plot(x,decoForest.pass_data_rate_lt[:], label="data-pass-rate")
plt.legend()

plt.show()

In [ ]:
# n=200, criterion="gini", dropout=None
np.max(np.array(decoForest.test_loss_lt)),list(enumerate(decoForest.test_loss_lt) )

In [ ]:
# n=50, criterion="gini", dropout=0.8
np.max(np.array(decoForest.test_loss_lt)),list(enumerate(decoForest.test_loss_lt) )

In [ ]:
# n=50, criterion="gini", dropout=None lift&support
np.max(np.array(decoForest.test_loss_lt)),list(enumerate(decoForest.test_loss_lt) )

In [ ]:
# n=50, criterion="gini", dropout=None lift
np.max(np.array(decoForest.test_loss_lt)),list(enumerate(decoForest.test_loss_lt) )

In [ ]:
# n=50, criterion="gini", dropout=None
np.max(np.array(decoForest.test_loss_lt)),list(enumerate(decoForest.test_loss_lt) )

In [ ]:
# n=200, criterion="gini", dropout=None
np.max(np.array(decoForest.test_loss_lt)),list(enumerate(decoForest.test_loss_lt) )

In [ ]:
# n=200, criterion="gini", dropout=-1
np.max(np.array(decoForest.test_loss_lt)),list(enumerate(decoForest.test_loss_lt) )

In [ ]:
# n=50, criterion="gini", dropout=-1
np.max(np.array(decoForest.test_loss_lt)),list(enumerate(decoForest.test_loss_lt) )

In [ ]:
# n=200, criterion="gini", dropout=None
np.max(np.array(decoForest.test_loss_lt)),list(enumerate(decoForest.test_loss_lt) )